In [273]:
import pandas as pd
import numpy as np
from lxml import etree
import json
import pdb

#USER AGENTS

In [330]:
df = load_data('data/products2.json')

In [384]:
def load_data(file_name):
    with open(file_name) as d:
        data = json.load(d)
        d.close()
    return pd.DataFrame(data)


def remove_punc_nan(df, col, punc):
    holder = []
    for item in df[col]:
        if not type(item) is float:
            item = item.strip(str(punc))
        holder.append(item)
    df[col] = holder
    return df

In [385]:
    # 1_star - DONE
    # 2_star - DONE
    # 3_star - DONE
    # 4_star - DONE
    # 5_star - DONE
    star_cols = ['1_star', '2_star', '3_star', '4_star', '5_star']
    for star in star_cols:
        remove_punc_nan(df, star, '%')

    # ASIN - WIP - Ignore/coerce to nan bad ASINs for now
    # Batteries - DONE - DONE
    # Best_Sellers_Rank - DONE
    df.drop(['Best_Sellers_Rank'], axis=1, inplace=True)
    # California_residents - DONE - DONE
    # Customer_Reviews -
    # Department -  - DONE
    # Domestic_Shipping - DONE
    df.replace(to_replace='Item can be shipped within U.S.', value={
               'Domestic_Shipping': 'Domestic'}, inplace=True)
    df.replace(to_replace='This item is not eligible for international \
                           shipping.', value={
               'Domestic_Shipping': 'Domestic'}, inplace=True)
    df.replace(to_replace='This item can be shipped to select countries outside\
                           of the U.S.', value={
               'Domestic_Shipping': 'International'}, inplace=True)
    df.replace(to_replace='Currently, item can be shipped only within the U.S. \
                            and to APO/FPO addresses. For APO/FPO shipments, \
                            please check with the manufacturer regarding \
                            warranty and support issues.',
               value={'Domestic_Shipping': 'Domestic'}, inplace=True)
    # International_Shipping - WIP--use: # (?=International)*(?=Domestic)*
    df.drop(['International_Shipping'], axis=1, inplace=True)
    # Item_Weight - DONE
    # Item_model_number - DONE
    # Manufacturer - DONE
    # Manufacturer_recommended_age - DONE
    # Media - DONE
    # Origin - DONE
    # Pricing - DONE
    # Product_Dimensions - DONE
    df.replace(to_replace='Currently, item can be shipped only within the U.S. \
                            and to APO/FPO addresses. For APO/FPO shipments, \
                            please check with the manufacturer regarding \
                            warranty and support issues.',
               value={'Product_Dimensions': np.nan}, inplace=True)
    df.replace(to_replace='Item can be shipped within U.S.',
               value={'Product_Dimensions': np.nan}, inplace=True)
    # Release_date - DONE
    # Shipping_Information - DONE
    df.drop(['Shipping_Information'], axis=1, inplace=True)
    # Shipping_Weight - DONE
    # about - DONE
    # avg_rating - DONE
    df.replace(to_replace=r'(?![0-9][\.])( out of 5 stars)',
               value={'avg_rating': ''}, inplace=True, regex=True)
    df.replace(to_replace='', value={'avg_rating': np.nan}, inplace=True)
    # category - DONE
    # description - DONE
    # in_stock - WIP
    df.replace(to_replace=r'(?![0-9])?(?=\s+left in stock.)*(Only\s+|\s+left in stock - order soon.\s+)+',
               value={'in_stock': ''}, inplace=True, regex=True)
    df.replace(to_replace=r'(?![0-9])?(\s+left in stock.)+',
               value={'in_stock': ''}, inplace=True, regex=True)
    # convert in stock/electronic goods to 9999
    df.replace(to_replace=r'(Sold by Amazon Appstore|In Stock\.|Available from these sellers\.)', value={
               'in_stock': '9999'}, inplace=True, regex=True)
    # list_price - DONE
    df['list_price'].fillna(value=0, inplace=True)
    df.replace(to_replace=r'\$', value={
               'list_price': ''}, inplace=True, regex=True)
    df['list_price'] = pd.to_numeric(df['list_price'], errors='ignore')
    # sale_price - DONE
    df['sale_price'].fillna(value=0, inplace=True)
    df.replace(to_replace=r'\$', value={
               'sale_price': ''}, inplace=True, regex=True)
    df['sale_price'] = pd.to_numeric(df['sale_price'], errors='ignore')
    # num_questions - DONE
    df.replace(to_replace=r'(?![0-9])(.)+',
               value={'num_questions': ''}, inplace=True, regex=True)
    df.replace(to_replace='', value={'num_questions': np.nan}, inplace=True)
    df['num_questions'] = pd.to_numeric(df['num_questions'], errors='ignore')
    # num_reviews - DONE
    df.replace(to_replace=r'(\,)', value={
               'num_reviews': ''}, inplace=True, regex=True)
    df['num_reviews'] = pd.to_numeric(df['num_reviews'], errors='ignore')
    # product_title - DONE
    df.replace(to_replace='', value={'product_title': np.nan}, inplace=True)
    # reviews_url - DONE
    # root_or_child - DONE
    # shipping - DONE
    # url - DONE

In [464]:
df_mod = df.loc[
        (~df['1_star'].isnull()) &
        (~df['2_star'].isnull()) &
        (~df['3_star'].isnull()) &
        (~df['4_star'].isnull()) &
        (~df['5_star'].isnull()) &
        (~df['ASIN'].isnull()), 
        ]

In [469]:
    urls_to_revisit = df.loc[
        (~df['1_star'].isnull()) &
        (~df['2_star'].isnull()) &
        (~df['3_star'].isnull()) &
        (~df['4_star'].isnull()) &
        (~df['5_star'].isnull()) &
        (~df['ASIN'].isnull()), 'url'
    ]

In [473]:
df_mod.columns

Index(['1_star', '2_star', '3_star', '4_star', '5_star', 'ASIN', 'Batteries',
       'California_residents', 'Customer_Reviews', 'Date_First_Available',
       'Department', 'Discontinued_by_manufacturer', 'Domestic_Shipping',
       'Item_Weight', 'Item_model_number', 'Manufacturer',
       'Manufacturer_recommended_age', 'Media', 'Origin', 'Pricing',
       'Product_Dimensions', 'Release_date', 'Shipping_Advisory',
       'Shipping_Weight', 'about', 'avg_rating', 'category', 'description',
       'list_price', 'num_questions', 'num_reviews', 'product_title',
       'reviews_url', 'root_or_child', 'sale_price', 'shipping', 'url'],
      dtype='object')

In [472]:
df_mod.to_csv('data/products_cleaned.csv')

In [470]:
urls_to_revisit.to_csv('data/products_toscrape.csv')